<a href="https://colab.research.google.com/github/amirghnt/AcademicProjects/blob/main/Proposal_v_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

!pip install openpyxl
!pip install pulp

import pulp

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
OrderList = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='OrderList')
FreightRates = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='FreightRates')
WhCosts = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='WhCosts')
WhCapacities = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='WhCapacities')
ProductsPerPlant = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='ProductsPerPlant')
VmiCustomers = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='VmiCustomers')
PlantPorts = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='PlantPorts')

In [7]:
# Data cleaning for OrderList

# OrderList format conversion
OrderList['TPT'] = OrderList['TPT'].astype(int)
OrderList['Ship ahead day count'] = OrderList['Ship ahead day count'].astype(int)
OrderList['Ship Late Day count'] = OrderList['Ship Late Day count'].astype(int)
OrderList['Unit quantity'] = OrderList['Unit quantity'].astype(int)
OrderList['Weight'] = OrderList['Weight'].astype(float)
# drop nulls form OrderList
OrderList.dropna(inplace=True)
# drop duplicates from OrderList
OrderList.drop_duplicates(inplace=True)

# Data cleaning for FreightRates

# FreightRates format conversion
FreightRates['minm_wgh_qty'] = FreightRates['minm_wgh_qty'].astype(int)
FreightRates['max_wgh_qty'] = FreightRates['max_wgh_qty'].astype(float)
FreightRates['minimum cost'] = FreightRates['minimum cost'].astype(float)
FreightRates['rate'] = FreightRates['rate'].astype(float)
FreightRates['tpt_day_cnt'] = FreightRates['tpt_day_cnt'].astype(int)
# drop nulls form FreightRates
FreightRates.dropna(inplace=True)
# drop duplicates from FreightRates
FreightRates.drop_duplicates(inplace=True)

# Data cleaning for WhCosts

# WhCosts format conversion
WhCosts['Cost/unit'] = WhCosts['Cost/unit'].astype(float)
# drop nulls form WhCosts
WhCosts.dropna(inplace=True)
# drop duplicates from WhCosts
WhCosts.drop_duplicates(inplace=True)

# Data cleaning for WhCapacities

# WhCapacities format conversion
WhCapacities['Daily Capacity '] = WhCapacities['Daily Capacity '].astype(int)
# drop nulls form WhCapacities
WhCapacities.dropna(inplace=True)
# drop duplicates from WhCapacities
WhCapacities.drop_duplicates(inplace=True)

# Data cleaning for ProductsPerPlant

# drop nulls form ProductsPerPlant
ProductsPerPlant.dropna(inplace=True)
# drop duplicates from ProductsPerPlant
ProductsPerPlant.drop_duplicates(inplace=True)

# Data cleaning for VmiCustomers

# drop nulls form VmiCustomers
VmiCustomers.dropna(inplace=True)
# drop duplicates from VmiCustomers
VmiCustomers.drop_duplicates(inplace=True)

# Data cleaning for

# drop nulls form PlantPorts
PlantPorts.dropna(inplace=True)
# drop duplicates from PlantPorts
PlantPorts.drop_duplicates(inplace=True)

In [8]:
print(OrderList.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9215 entries, 0 to 9214
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order ID              9215 non-null   float64       
 1   Order Date            9215 non-null   datetime64[ns]
 2   Origin Port           9215 non-null   object        
 3   Carrier               9215 non-null   object        
 4   TPT                   9215 non-null   int64         
 5   Service Level         9215 non-null   object        
 6   Ship ahead day count  9215 non-null   int64         
 7   Ship Late Day count   9215 non-null   int64         
 8   Customer              9215 non-null   object        
 9   Product ID            9215 non-null   int64         
 10  Plant Code            9215 non-null   object        
 11  Destination Port      9215 non-null   object        
 12  Unit quantity         9215 non-null   int64         
 13  Weight            

In [9]:
print(FreightRates.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1539
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Carrier       1537 non-null   object 
 1   orig_port_cd  1537 non-null   object 
 2   dest_port_cd  1537 non-null   object 
 3   minm_wgh_qty  1537 non-null   int64  
 4   max_wgh_qty   1537 non-null   float64
 5   svc_cd        1537 non-null   object 
 6   minimum cost  1537 non-null   float64
 7   rate          1537 non-null   float64
 8   mode_dsc      1537 non-null   object 
 9   tpt_day_cnt   1537 non-null   int64  
 10  Carrier type  1537 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 144.1+ KB
None


In [10]:
print(WhCosts.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WH         19 non-null     object 
 1   Cost/unit  19 non-null     float64
dtypes: float64(1), object(1)
memory usage: 456.0+ bytes
None


In [11]:
print(WhCapacities.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Plant ID         19 non-null     object
 1   Daily Capacity   19 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 456.0+ bytes
None


In [12]:
print(ProductsPerPlant.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2036 entries, 0 to 2035
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  2036 non-null   object
 1   Product ID  2036 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.7+ KB
None


In [13]:
print(VmiCustomers.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  14 non-null     object
 1   Customers   14 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes
None


In [14]:
print(PlantPorts.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  22 non-null     object
 1   Port        22 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
None
